In [33]:
import pandas as pd
import json
import http.client
import asyncio

conn = http.client.HTTPSConnection("demo-api-capital.backend-capital.com")
identifier = "2211054370abc@gmail.com"
password = "Abc20030315;"
api_key = 'Ap4ukeVKKZY7aVpG'
payload = json.dumps({
    "identifier": identifier,
    "password": password
})
headers = {
    'X-CAP-API-KEY': api_key,
    'Content-Type': 'application/json'
}
conn.request("POST", "/api/v1/session", payload, headers)        
res = conn.getresponse()
sec_token = res.getheader('X-SECURITY-TOKEN')
cst = res.getheader('CST')
print(sec_token, cst)


eUcvQlDGmT7VwClOhaddHFAR5YnYHRa pXvjAhdR5kI6AB3QUCMuR2qz


In [34]:
conn = http.client.HTTPSConnection("demo-api-capital.backend-capital.com")
payload = ''
headers = {
  'X-SECURITY-TOKEN': sec_token,
  'CST': cst
}
conn.request("GET", "/api/v1/marketnavigation", payload, headers)
res = conn.getresponse()
data = json.loads(res.read().decode('utf-8'))
ids = [node['id'] for node in data['nodes']]
print(data)

{'nodes': [{'id': 'hierarchy_v1.commons_group', 'name': 'commons_group'}, {'id': 'hierarchy_v1.commodities_group', 'name': 'commodities_group'}, {'id': 'hierarchy_v1.oil_markets_group', 'name': 'oil_markets_group'}, {'id': 'hierarchy_v1.indices_group', 'name': 'indices_group'}, {'id': 'hierarchy_v1.forex', 'name': 'Forex'}, {'id': 'hierarchy_v1.crypto_currencies_group', 'name': 'crypto_currencies_group'}, {'id': 'hierarchy_v1.shares', 'name': 'Shares'}, {'id': 'hierarchy_v1.etf_group', 'name': 'etf_group'}]}


In [40]:
def jsonify_data(response):
    return json.loads(response.read().decode('utf-8'))

def get_tickers(ids:list[str], sec_token:str, cst:str, epics:set[str]) -> list[str]:
    conn = http.client.HTTPSConnection("api-capital.backend-capital.com")
    payload = ''
    headers = {
      'X-SECURITY-TOKEN': sec_token,
      'CST': cst
    }
    for id in ids:
        conn.request("GET", f"/api/v1/marketnavigation/{id}", payload, headers)
        res = conn.getresponse()
        data = jsonify_data(res)
        if 'markets' not in data.keys():
            new_ids:list[str] = [node['id'] for node in data['nodes']]
            get_tickers(new_ids, sec_token, cst, epics)
        else:
            epics.update(set([node['epic'] for node in data['markets']]))
            break
    return list(epics)

def get_overnight_rates(epics:list[str], sec_token:str, cst:str):
    conn = http.client.HTTPSConnection("demo-api-capital.backend-capital.com")
    payload = ''
    headers = {
      'X-SECURITY-TOKEN': sec_token,
      'CST': cst
    }
    res = []
    print("fetching ticker info...")
    for epic in epics:
        conn.request("GET", f"/api/v1/markets/{epic}", payload, headers)
        res.append(jsonify_data(conn.getresponse()))
    return res


In [31]:
epics = set()
print("fetching tickers...")
get_tickers(ids, sec_token, cst, epics)

fetching tickers...
fetching ticker info...


[{'instrument': {'epic': 'SPCE',
   'symbol': 'SPCE',
   'expiry': '-',
   'name': 'Virgin Galactic Holdings Inc (Extended Hours)',
   'lotSize': 1,
   'type': 'SHARES',
   'guaranteedStopAllowed': True,
   'streamingPricesAvailable': True,
   'currency': 'USD',
   'marginFactor': 20,
   'marginFactorUnit': 'PERCENTAGE',
   'openingHours': {'mon': ['08:00 - 00:00'],
    'tue': ['08:00 - 00:00'],
    'wed': ['08:00 - 00:00'],
    'thu': ['08:00 - 00:00'],
    'fri': ['08:00 - 21:00'],
    'sat': [],
    'sun': [],
    'zone': 'UTC'},
   'country': 'United States of America',
   'overnightFee': {'longRate': -0.0262347,
    'shortRate': 0.0040124,
    'swapChargeTimestamp': 1723755600000,
    'swapChargeInterval': 1440}},
  'dealingRules': {'minStepDistance': {'unit': 'POINTS', 'value': 0.0001},
   'minDealSize': {'unit': 'POINTS', 'value': 1},
   'maxDealSize': {'unit': 'POINTS', 'value': 26700},
   'minSizeIncrement': {'unit': 'POINTS', 'value': 1},
   'minGuaranteedStopDistance': {'uni

In [41]:
data = get_overnight_rates(epics, sec_token, cst)
data

fetching ticker info...


[{'instrument': {'epic': 'SPCE',
   'symbol': 'SPCE',
   'expiry': '-',
   'name': 'Virgin Galactic Holdings Inc (Extended Hours)',
   'lotSize': 1,
   'type': 'SHARES',
   'guaranteedStopAllowed': True,
   'streamingPricesAvailable': True,
   'currency': 'USD',
   'marginFactor': 20,
   'marginFactorUnit': 'PERCENTAGE',
   'openingHours': {'mon': ['08:00 - 00:00'],
    'tue': ['08:00 - 00:00'],
    'wed': ['08:00 - 00:00'],
    'thu': ['08:00 - 00:00'],
    'fri': ['08:00 - 21:00'],
    'sat': [],
    'sun': [],
    'zone': 'UTC'},
   'country': 'United States of America',
   'overnightFee': {'longRate': -0.0262347,
    'shortRate': 0.0040124,
    'swapChargeTimestamp': 1723755600000,
    'swapChargeInterval': 1440}},
  'dealingRules': {'minStepDistance': {'unit': 'POINTS', 'value': 0.0001},
   'minDealSize': {'unit': 'POINTS', 'value': 1},
   'maxDealSize': {'unit': 'POINTS', 'value': 26700},
   'minSizeIncrement': {'unit': 'POINTS', 'value': 1},
   'minGuaranteedStopDistance': {'uni

In [1]:
import csv
df = pd.DataFrame(data)
df = df.drop("errorCode", axis=1)
instruments = df['instrument'].dropna()
instruments.to_json("instruments_data.json", orient='records')


NameError: name 'pd' is not defined

In [8]:
import pandas as pd
instruments = pd.read_json("instruments_data.json")
list(instruments.values)
info = [(node[0], node[5], node[-1]['longRate'], node[-1]['shortRate']) for node in list(instruments.values)]
info1 = sorted(info, key=lambda x: x[2], reverse=True)
info2 = sorted(info, key=lambda x: x[3], reverse=True)
df1 = pd.DataFrame(info1, columns=['epic', 'type', 'longRate', 'shortRate'])
df1['annualised longRate'] = (1 + df1['longRate']) ** 365
df1['annualised shortRate'] = (1 + df1['shortRate']) ** 365
df2 = pd.DataFrame(info2, columns=['epic', 'type', 'longRate', 'shortRate'])
df2['annualised shortRate'] = (1 + df1['shortRate']) ** 365
df2['annualised longRate'] = (1 + df1['longRate']) ** 365
df1


,epic,type,longRate,shortRate,annualised longRate,annualised shortRate
0,USCOCOA,COMMODITIES,0.422219,-0.444139,6.804167e+55,8.177792e-94
1,VIX,INDICES,0.112345,-0.134265,7.539856e+16,1.398264e-23
2,OIL_CRUDE,COMMODITIES,0.056456,-0.078376,5.078157e+08,1.153976e-13
3,MXNJPY,CURRENCIES,0.026530,-0.034750,1.414656e+04,2.474747e-06
4,ORANGEJUICE,COMMODITIES,0.021558,-0.043478,2.404566e+03,8.987283e-08
...,...,...,...,...,...,...
560,PEPEUSD,CRYPTOCURRENCIES,-0.137000,0.006850,4.404944e-24,1.208211e+01
561,LUNCUSD,CRYPTOCURRENCIES,-0.137000,0.006850,4.404944e-24,1.208211e+01
562,EOSUSD,CRYPTOCURRENCIES,-0.137000,0.006850,4.404944e-24,1.208211e+01
563,APEUSD,CRYPTOCURRENCIES,-0.137000,0.006850,4.404944e-24,1.208211e+01


In [9]:
df2

,epic,type,longRate,shortRate,annualised shortRate,annualised longRate
0,NATURALGAS,COMMODITIES,-0.194930,0.173010,8.177792e-94,6.804167e+55
1,CORN,COMMODITIES,-0.087353,0.065433,1.398264e-23,7.539856e+16
2,BTCEUR,CRYPTOCURRENCIES,-0.061640,0.013700,1.153976e-13,5.078157e+08
3,ETHEUR,CRYPTOCURRENCIES,-0.061640,0.013700,2.474747e-06,1.414656e+04
4,BTCUSD,CRYPTOCURRENCIES,-0.061640,0.013700,8.987283e-08,2.404566e+03
...,...,...,...,...,...,...
560,OIL_BRENT,COMMODITIES,0.021240,-0.043160,1.208211e+01,4.404944e-24
561,ORANGEJUICE,COMMODITIES,0.021558,-0.043478,1.208211e+01,4.404944e-24
562,OIL_CRUDE,COMMODITIES,0.056456,-0.078376,1.208211e+01,4.404944e-24
563,VIX,INDICES,0.112345,-0.134265,1.208211e+01,4.404944e-24


In [12]:
df1.groupby('type')['longRate'].mean()

type
COMMODITIES         0.009549
CRYPTOCURRENCIES   -0.095215
CURRENCIES          0.003200
INDICES            -0.012212
SHARES             -0.023373
Name: longRate, dtype: float64

In [133]:
df1.groupby('type')['longRate'].max()

type
COMMODITIES         0.422219
CRYPTOCURRENCIES   -0.026235
CURRENCIES          0.026530
INDICES             0.112345
SHARES             -0.014317
Name: longRate, dtype: float64

In [6]:
df1.groupby('type')['shortRate'].max()

type
COMMODITIES         0.173010
CRYPTOCURRENCIES    0.013700
CURRENCIES          0.000375
INDICES             0.004012
SHARES              0.004012
Name: shortRate, dtype: float64

In [18]:
df1.loc[df1['type']=='CRYPTOCURRENCIES'].sort_values(by='longRate', ascending=False)

,epic,type,longRate,shortRate,annualised longRate,annualised shortRate
498,OMGUSD,CRYPTOCURRENCIES,-0.026235,0.004012,6.106853e-05,4.312836
532,BTCEUR,CRYPTOCURRENCIES,-0.061640,0.013700,8.219826e-11,143.531747
533,ETHEUR,CRYPTOCURRENCIES,-0.061640,0.013700,8.219826e-11,143.531747
534,BTCUSD,CRYPTOCURRENCIES,-0.061640,0.013700,8.219826e-11,143.531747
535,ETHUSD,CRYPTOCURRENCIES,-0.061640,0.013700,8.219826e-11,143.531747
536,LTCEUR,CRYPTOCURRENCIES,-0.075340,0.006850,3.832268e-13,12.082106
537,BCHUSD,CRYPTOCURRENCIES,-0.075340,0.006850,3.832268e-13,12.082106
538,C10X,CRYPTOCURRENCIES,-0.075340,0.006850,3.832268e-13,12.082106
539,SOLUSD,CRYPTOCURRENCIES,-0.075340,0.006850,3.832268e-13,12.082106
540,ADAUSD,CRYPTOCURRENCIES,-0.075340,0.006850,3.832268e-13,12.082106


In [16]:
df1.loc[df1['type']=='INDICES'].sort_values(by='longRate', ascending=False)

,epic,type,longRate,shortRate,annualised longRate,annualised shortRate
1,VIX,INDICES,0.112345,-0.134265,7.539856e+16,1.398264e-23
37,DXY,INDICES,-0.007399,-0.014521,6.649187e-02,4.800556e-03
41,J225,INDICES,-0.011249,-0.010973,1.609413e-02,1.782338e-02
67,SW20,INDICES,-0.014317,-0.007905,5.176903e-03,5.520014e-02
68,CN50,INDICES,-0.016030,-0.005890,2.743961e-03,1.157635e-01
77,FR40,INDICES,-0.021418,-0.000804,3.697752e-04,7.456183e-01
84,EU50,INDICES,-0.021418,-0.000804,3.697752e-04,7.456183e-01
92,DE40,INDICES,-0.021418,-0.000804,3.697752e-04,7.456183e-01
95,SP35,INDICES,-0.021418,-0.000804,3.697752e-04,7.456183e-01
175,NL25,INDICES,-0.021418,-0.000804,3.697752e-04,7.456183e-01


In [127]:
import matplotlib.pyplot as plt
import numpy as np
longRate = 0.0137
(1 + longRate/100)**np.arange(365)

array([1.        , 1.000137  , 1.00027402, 1.00041106, 1.00054811,
       1.00068519, 1.00082228, 1.00095939, 1.00109653, 1.00123368,
       1.00137084, 1.00150803, 1.00164524, 1.00178246, 1.00191971,
       1.00205697, 1.00219425, 1.00233155, 1.00246887, 1.00260621,
       1.00274357, 1.00288094, 1.00301834, 1.00315575, 1.00329319,
       1.00343064, 1.00356811, 1.0037056 , 1.0038431 , 1.00398063,
       1.00411817, 1.00425574, 1.00439332, 1.00453092, 1.00466854,
       1.00480618, 1.00494384, 1.00508152, 1.00521922, 1.00535693,
       1.00549467, 1.00563242, 1.00577019, 1.00590798, 1.00604579,
       1.00618362, 1.00632147, 1.00645933, 1.00659722, 1.00673512,
       1.00687304, 1.00701098, 1.00714894, 1.00728692, 1.00742492,
       1.00756294, 1.00770098, 1.00783903, 1.0079771 , 1.0081152 ,
       1.00825331, 1.00839144, 1.00852959, 1.00866776, 1.00880595,
       1.00894415, 1.00908238, 1.00922062, 1.00935888, 1.00949717,
       1.00963547, 1.00977379, 1.00991213, 1.01005049, 1.01018